In [ ]:
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as ct
from scipy.interpolate import interp1d
import trimesh as tm
import trimesh.ray.ray_pyembree
import h5py
import pyiron.vasp.structure
from mpl_toolkits.mplot3d import Axes3D
import time
import seaborn as sns

np.set_printoptions(precision=3)

print('done importing modules!')

In [ ]:
mesh = tm.load('std_geo/cuboid.stl') # shape2S2

# manipulating mesh
mesh.vertices /= 2
mesh.rezero()
mesh.vertices = mesh.vertices*np.array([5, 1, 1])   # resizing cube
limits = mesh.bounds

phonons = h5py.File('kappa-m20206.hdf5', 'r')                   # phonon properties
crystal = pyiron.vasp.structure.read_atoms('POSCAR-unitcell')   # crystal properties

# crystal.plot3d()
# mesh.show()

In [ ]:
# PHONON MODES COMPUTATION

crystal_omega = np.array(phonons['frequency'])*2*ct.pi*1e12

n_of_qpoints = np.array(phonons['frequency']).shape[0]
n_of_branches = np.array(phonons['frequency']).shape[1]
n_of_modes = n_of_branches*n_of_qpoints

T_c = 1    # cold temperature
T_h = 500   # hot temperature
T_step = 1 # step of temperature array

crystal_temperature = np.arange(T_c, T_h+2*T_step, T_step)
crystal_energy      = np.zeros(crystal_temperature.shape)

for i in range(crystal_temperature.shape[0]):
    T = crystal_temperature[i]
    crystal_occupation = 1/(np.exp(ct.hbar*crystal_omega/(ct.k*T))-1)
    
    E = ct.hbar*crystal_omega*(0.5 + crystal_occupation)
    crystal_energy[i] = E.sum()/n_of_modes   # average energy per particle at correspondent temperature

energy_function = interp1d(crystal_temperature, crystal_energy) # average energy per particle function e = f(T)
temperature_function = interp1d(crystal_energy, crystal_temperature) # temperature function e = f(T)


# COMPUTE PRIORITIES

matrix = np.abs(crystal_omega)

rank = matrix.argsort(axis = None).argsort().reshape(matrix.shape)

fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot(121)
ax = sns.heatmap(crystal_omega, cmap = 'viridis')
ax.invert_yaxis()
ax = fig.add_subplot(122)
ax = sns.heatmap(rank,  cmap = 'viridis')
ax.invert_yaxis()


In [ ]:
# DIVIDE PARTICLES FOR EACH MODE

n_of_particles = int(1*n_of_modes)

part_p_mode = np.ceil(n_of_particles/n_of_modes)    # particles per mode

surplus = int(part_p_mode*n_of_modes-n_of_particles)    # excess of particles due to rounding

surplus_modes = np.where(rank.reshape(-1)>(rank.max()-surplus))[0]

ppm_array = (np.ones(n_of_modes)*part_p_mode).astype(int)

ppm_array[surplus_modes] -= 1                                   # number of particles per mode without excess

In [ ]:
# PARTICLE GENERATION (LAURENT'S METHOD)

start = time.time()

modes     = np.zeros( (1, 2) )
positions = np.zeros( (1, 3) )

for i in range(n_of_qpoints):
    for j in range(n_of_branches):

        index = int(i*n_of_branches+j)

        buffer_modes      = np.ones( (ppm_array[index], 2) )
        buffer_modes[:,0] *= i                         
        buffer_modes[:,1] *= j

        buffer_positions = np.random.rand(buffer_modes.shape[0], 3)*(limits[1,:] - limits[0,:]) + limits[0,:]
        in_points = mesh.contains(buffer_positions)
        buffer_positions = np.delete(buffer_positions, ~in_points, axis=0)

        new_points = np.random.rand(buffer_modes.shape[0]-buffer_positions.shape[0], 3)*(limits[1,:] - limits[0,:]) + limits[0,:]
        in_points = mesh.contains(new_points)

        buffer_positions = np.row_stack( (buffer_positions, new_points[in_points]) )

        while np.any(~in_points):
            new_points = np.random.rand(buffer_modes.shape[0]-buffer_positions.shape[0], 3)*(limits[1,:] - limits[0,:]) + limits[0,:]

            in_points = mesh.contains(new_points)

            buffer_positions = np.row_stack( (buffer_positions, new_points[in_points]) )

        modes = np.row_stack( (modes, buffer_modes) )

        positions = np.row_stack( (positions, buffer_positions) )

        print('Mode {:>4d} of {:>4d}, Q-point {:>3d}, branch {:>2d}: {:>10d} particles.'.format(index+1, n_of_modes, i, j, ppm_array[index]))

modes     = np.delete(modes, 0, axis = 0).astype(int)
positions = np.delete(positions, 0, axis = 0)

end = time.time()

print('Particles generated! Time: {:.3f} s'.format(end-start))



In [ ]:
# PARTICLE GENERATION (BRUNO'S METHOD)

start = time.time()

modes = np.zeros( (1, 2) )

for i in range(n_of_qpoints):
    for j in range(n_of_branches):

        index = int(i*n_of_branches+j)

        buffer_modes      = np.ones( (ppm_array[index], 2) )
        buffer_modes[:,0] *= i                         
        buffer_modes[:,1] *= j

        modes = np.row_stack( (modes, buffer_modes) )

modes     = np.delete(modes, 0, axis = 0).astype(int)

positions = np.random.rand(n_of_particles, 3)*(limits[1,:] - limits[0,:]) + limits[0,:]
in_points = mesh.contains(positions)

positions = np.delete(positions, ~in_points, axis=0)
new_points = np.random.rand(modes.shape[0]-positions.shape[0], 3)*(limits[1,:] - limits[0,:]) + limits[0,:]

in_points = mesh.contains(new_points)

positions = np.row_stack( (positions, new_points[in_points, :]) )

while np.any(~in_points):
    new_points = np.random.rand(modes.shape[0]-positions.shape[0], 3)*(limits[1,:] - limits[0,:]) + limits[0,:]

    in_points = mesh.contains(new_points)

    positions = np.row_stack( (positions, new_points[in_points, :]) )

    # print('out particles {}'.format(~in_points.sum()))
    

end = time.time()

print('Particles generated! Time: {:.3f} s'.format(end-start))

# fig = plt.figure(figsize = (15, 5))
# for i in range(15):
#     points = positions[ int(i*part_p_mode):int((i+1)*part_p_mode), :]
#     ax = fig.add_subplot(3, 5, i+1, projection='3d')
#     ax.set_box_aspect( (5, 1, 1) )
#     ax.plot(points[:,0], points[:,1], points[:,2], '.')
# plt.tight_layout()

In [ ]:
# FREQUENCY ASSIGNMENT

# modes = np.zeros( (n_of_particles, 2) )

# modes[:,0] = np.floor(np.random.rand(n_of_particles)*n_of_qpoints)
# modes[:,1] = np.floor(np.random.rand(n_of_particles)*n_of_branches)

# modes = modes.astype(int)

frequencies = np.abs( crystal_omega[modes[:,0], modes[:,1]] )

velocities = np.array(phonons['group_velocity'])[modes[:,0], modes[:,1], :]



In [ ]:
# SLICING

n_of_slices = 10
slice_axis  = 0

slice_size = np.ptp(limits[:,slice_axis])/n_of_slices

T_c = 20
T_h = 50

T_array = np.arange(n_of_slices)*(T_h - T_c)/(n_of_slices-1)+T_c
T_array = np.flip(T_array)

temperatures = np.zeros(frequencies.shape)

particles_per_slice = np.zeros(n_of_slices)

# fig = plt.figure(figsize = (15, 5))
# ax = fig.add_subplot(111, projection='3d')
# ax.set_box_aspect( (5, 1, 1) )

sliced_modes_data = []

for i in range(n_of_slices): 

    indexes = (positions[:, slice_axis] >= i*slice_size) & (positions[:, slice_axis] < (i+1)*slice_size)
    sliced_points = positions[indexes, :]
    sliced_modes_data += [modes[indexes, :]]
    particles_per_slice[i] =sliced_points.shape[0]
    
    temperatures[indexes] = T_array[i]
    
occupation = 1/( np.exp( ct.hbar*frequencies/(ct.k*temperatures) ) - 1)
energies = ct.hbar*frequencies*(occupation+0.5)

indexes = np.arange( n_of_particles )#((modes[:,0] == 0) & (modes[:,1] == 0)) | ((modes[:,0] == 0) & (modes[:,1] == 15))
points = positions[indexes, :]

fig = plt.figure(figsize = (15, 15))

ax = fig.add_subplot(221, projection='3d')
ax.set_box_aspect( np.ptp(limits, axis = 0) )
colors = frequencies[indexes]
graph = ax.scatter(points[:,0], points[:,1], points[:,2], c = colors, cmap = 'viridis', s = 2 )
fig.colorbar(graph, orientation = 'horizontal')
ax.set_title('Frequencies [rad/s]')


ax = fig.add_subplot(222, projection='3d')
ax.set_box_aspect( np.ptp(limits, axis = 0) )
colors = temperatures[indexes]
graph = ax.scatter(points[:,0], points[:,1], points[:,2], c = colors, cmap = 'viridis', s = 2 )
fig.colorbar(graph, orientation = 'horizontal')
ax.set_title('Temperatures [K]')

ax = fig.add_subplot(223, projection='3d')
ax.set_box_aspect( np.ptp(limits, axis = 0) )
colors = energies[indexes]
graph = ax.scatter(points[:,0], points[:,1], points[:,2], c = colors, cmap = 'viridis', s = 2 )
fig.colorbar(graph, orientation = 'horizontal')
ax.set_title('Energies [J]')

ax = fig.add_subplot(224, projection='3d')
ax.set_box_aspect( np.ptp(limits, axis = 0) )
colors = np.log(occupation[indexes])
graph = ax.scatter(points[:,0], points[:,1], points[:,2], c = colors, cmap = 'viridis', s = 2 )
fig.colorbar(graph, orientation = 'horizontal')
ax.set_title('Occupation number')

# plt.tight_layout()

In [ ]:
# HISTOGRAMS

fig = plt.figure(figsize = (15, 10))

# ax = fig.add_subplot(121)
# ax.bar(np.arange(n_of_slices), particles_per_slice)
# ax.set_title('Particles per slice.')
# ax.set_xlabel('Slice')

bins_data = []

for i in range(n_of_slices):
    data = sliced_modes_data[i]
    ax = fig.add_subplot(3, 4, i+1)
    bins, _, _, _ = ax.hist2d(data[:,1], data[:,0], bins = [n_of_branches, n_of_qpoints])
    ax.set_title('Particles per mode, slice {:>d} \n {:>d} particles, mean = {:.2f}, stdev = {:.2f}'.format(i, data.shape[0], data.mean(), data.std()),
                 fontsize = 10)
    ax.set_xlabel('Q-point')
    ax.set_ylabel('Branch')
    print( '{:>6d} particles; Mean particles per mode = {:>5.2f}; Std dev = {:>5.2f}; Max = {:>d}; Min = {:>d} '.format(data.shape[0], bins.mean(), bins.std(), bins.max().astype(int), bins.min().astype(int) ) )
    bins_data += [bins]

plt.tight_layout()




In [ ]:
bins_data = [i.reshape(-1) for i in bins_data]

fig = plt.figure(figsize = (8, 5))
ax = fig.add_subplot(111)
ax.boxplot(bins_data, whis = 'range')
ax.set_xlabel('Slice')
ax.set_ylabel('Mode incidence')
ax.set_ylim([np.ceil(min(bins_data[:])/10)*10, np.ceil(max(bins_data[:])/10)*10])

plt.tight_layout()

In [ ]:
# CALCULATING AVERAGES

energy_array = np.zeros(T_array.shape)
en_std_array = np.zeros(T_array.shape)
sliced_data = []

for i in range(n_of_slices):
    indexes = (positions[:, slice_axis] >= i*slice_size) & (positions[:, slice_axis] < (i+1)*slice_size)
    
    sliced_energies = energies[indexes]
    energy_array[i]  = sliced_energies.mean()   # average energy per particle
    en_std_array[i]  = sliced_energies.std()
    
    sliced_occupation = occupation[indexes]

    sliced_data += [sliced_occupation]

slice_array = np.arange(0, n_of_slices)

fig = plt.figure(figsize = (10, 5))
ax = fig.add_subplot(121)
# ax.boxplot(sliced_data, flierprops = {'marker': '.', 'markersize': 1, 'color': 'r'}, positions = slice_array )
ax.plot(slice_array, energy_array)
ax.set_title('Average energy per particle [J]')
ax.set_xlabel('Slice')
ax.set_xticks(np.arange(0, n_of_slices, 2))

ax = fig.add_subplot(122)
ax.plot(slice_array, T_array, '+', color = 'k', markersize = 10)

T_array_calc = temperature_function(energy_array)
ax.plot(slice_array, T_array_calc, '--')
ax.set_title('Average temperature [K]')
ax.set_xlabel('Slice')
ax.set_xticks(np.arange(0, n_of_slices, 2))
ax.legend(labels = ['T prescribed', 'T calculated'])

plt.tight_layout()



In [ ]:
# DRIFT OPERATION

dt = 0.01

start = time.time()

new_positions = positions+velocities*dt

end = time.time()

print('Drift done! Time = {:>.3f} s'.format(end-start))

start = time.time()

in_points = mesh.contains(new_positions)

end = time.time()

print('Check inside done! Time = {:>.3f} s'.format(end-start))

start = time.time()

new_positions = np.delete(new_positions, ~in_points, axis = 0)

end = time.time()

print('Delete out points done! Time = {:>.3f} s'.format(end-start))

start = time.time()

for i in range(n_of_slices): 

    indexes = (positions[:, slice_axis] >= i*slice_size) & (positions[:, slice_axis] < (i+1)*slice_size)
    energy_array[i] = energies[indexes].mean()

T_array_calc = temperature_function(energy_array)

end = time.time()

print('Updated energies and temperatures done! Time = {:>.3f} s'.format(end-start))



In [ ]:
# checking boundary periodicity computation

bounds = np.array([[0, 0, 0],
                   [1, 2, 1]])

points = np.random.rand(10, 3)*3-0.75

check = points >= bounds.reshape(2, 1, 3)

check = check.sum(axis = 0)

print(points)

lower_points = np.where(check == 0, points+(bounds[1, :]-bounds[0, :]), 0)
in_points = np.where(check == 1, points, 0)
upper_points = np.where(check == 2, points-(bounds[1, :]-bounds[0, :]), 0)

new_points = lower_points+in_points+upper_points

print(new_points)
print(check)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

X = np.random.rand(100, 3)*10
Y = np.random.rand(100, 3)*5

plt.ion()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(X[:, 0], X[:, 1], X[:, 2])
fig.show()

for i in range(0, 20):
    plt.pause(1)

    Y = np.random.rand(100, 3)*5

    sc._offsets3d = (Y[:,0], Y[:,1], Y[:,2])
    plt.draw()